**Task: File Ingestion and Schema validation**

Take any csv/text file of 2+ GB of your choice.

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of
read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

*  Total number of rows
*   total number of columns
*   file size



# Amazon Books Reviews dataset used from kaggle 
# (10 columns)

In [1]:
import os
import time

# Size of the file

In [2]:
os.path.getsize('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv')

2859504349

**Read the date with Pandas**

In [3]:
import pandas as pd

start = time.time()
pd_data = pd.read_csv('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv')
end = time.time()

print("Read data with Pandas: ",(end-start),"sec")


Read data with Pandas:  55.97093319892883 sec


**Read the data with Dask** 

In [4]:
from dask import dataframe as dd

start = time.time()
dask_data = dd.read_csv('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv')
end = time.time()

print("Read data with Dask: ",(end-start),"sec")


Read data with Dask:  0.006974697113037109 sec


**Read the data with Modin and Ray**

In [5]:
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
import modin.pandas as pd

start = time.time()
modin_data = pd.read_csv('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv')
end = time.time()

print("Read data with Modin: ",(end-start),"sec")



To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()



Read data with Modin:  17.882729053497314 sec


# Dask is better than Pandas, Modin and ray with least reading time of 0.006

In [6]:
from dask import dataframe as dd
d_data= dd.read_csv('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv')


In [7]:
d_data.info()


<class 'dask.dataframe.core.DataFrame'>
Columns: 10 entries, Id to review/text
dtypes: object(6), float64(2), int64(2)

**remove underscores**

In [8]:

d_data.columns=d_data.columns.str.replace('[_,@,&]','')



In [9]:
#To remove white space from columns
d_data.columns = d_data.columns.str.replace(' ', '')

In [10]:
d_data.columns

Index(['Id', 'Title', 'Price', 'Userid', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

# Validation

**Create YAML file**

**File Reading**

In [11]:
%%writefile Testutility.py
import yaml
import logging
import subprocess
import yaml
import datetime
import gc
import re 
import pandas as pd
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string


def col_header_val(df, table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file", mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded", missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting Testutility.py


In [12]:
%%writefile books.yaml
file_type: csv
dataset_name: file
file_name: books_rating
table_name : edsurv
inbound_delimiter : ","
outbound_delimiter : "|"
skip_leading_rows: 1
columns: 
    - Id
    - Title
    - Price
    - Userid
    - profileName
    - review/helpfulness
    - review/score
    - review/time
    - review/summary
    - review/text


Overwriting books.yaml


**Read config file**

In [13]:
import Testutility as util

data_config =util.read_config_file("books.yaml")

**Data of config file**

In [14]:

data_config

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'books_rating',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Id',
  'Title',
  'Price',
  'Userid',
  'profileName',
  'review/helpfulness',
  'review/score',
  'review/time',
  'review/summary',
  'review/text']}

**Normal reading process of the file**

In [20]:

from dask import dataframe as dd
df_data= dd.read_csv('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv',dtype={'Id': 'object'})
df_data.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


**Reading the file using config file**

In [25]:

file_type = data_config['file_type']
source_file = "./" + data_config['file_name'] + f'.{file_type}'
d_data = pd.read_csv(source_file,data_config['inbound_delimiter'])
d_data.head()



,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


**Validate the header of the file**

In [26]:

util.col_header_val(d_data,data_config)


column name and column length validation failed
Following File columns are not in the YAML file ['user_id', 'review_helpfulness', 'review_score', 'review_text', 'review_summary', 'review_time']
Following YAML columns are not in the file uploaded ['review/time', 'review/score', 'review/summary', 'review/text', 'userid', 'review/helpfulness']


0

In [27]:
print("columns of files are:" ,d_data.columns)
print("columns of YAML are:" ,data_config['columns'])

columns of files are: Index(['id', 'title', 'price', 'user_id', 'profilename', 'review_helpfulness',
       'review_score', 'review_time', 'review_summary', 'review_text'],
      dtype='object')
columns of YAML are: ['Id', 'Title', 'Price', 'Userid', 'profileName', 'review/helpfulness', 'review/score', 'review/time', 'review/summary', 'review/text']


In [28]:
if util.col_header_val(d_data,data_config)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['user_id', 'review_helpfulness', 'review_score', 'review_text', 'review_summary', 'review_time']
Following YAML columns are not in the file uploaded ['review/time', 'review/score', 'review/summary', 'review/text', 'userid', 'review/helpfulness']
validation failed


In [30]:
import csv
import gzip
import datetime

from dask import dataframe as dd
df = dd.read_csv('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv',delimiter=',',dtype={'Id': 'object'})


# Write csv in gz format in pipe separated text file (|)
df.to_csv('Books_rating.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['e:\\solo projects\\Data_Glacier_virtual_internship\\Data_Glacier_virtual_internship\\Week 6\\Books_rating.csv.gz\\00.part',
 'e:\\solo projects\\Data_Glacier_virtual_internship\\Data_Glacier_virtual_internship\\Week 6\\Books_rating.csv.gz\\01.part',
 'e:\\solo projects\\Data_Glacier_virtual_internship\\Data_Glacier_virtual_internship\\Week 6\\Books_rating.csv.gz\\02.part',
 'e:\\solo projects\\Data_Glacier_virtual_internship\\Data_Glacier_virtual_internship\\Week 6\\Books_rating.csv.gz\\03.part',
 'e:\\solo projects\\Data_Glacier_virtual_internship\\Data_Glacier_virtual_internship\\Week 6\\Books_rating.csv.gz\\04.part',
 'e:\\solo projects\\Data_Glacier_virtual_internship\\Data_Glacier_virtual_internship\\Week 6\\Books_rating.csv.gz\\05.part',
 'e:\\solo projects\\Data_Glacier_virtual_internship\\Data_Glacier_virtual_internship\\Week 6\\Books_rating.csv.gz\\06.part',
 'e:\\solo projects\\Data_Glacier_virtual_internship\\Data_Glacier_virtual_internship\\Week 6\\Books_rating.csv.gz\\07

**number of files in gz format folder**

**size of the gz format folder**

In [31]:

partitions = os.listdir('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv.gz')
for partition in partitions:
    print(partition)


00.part
01.part
02.part
03.part
04.part
05.part
06.part
07.part
08.part
09.part
10.part
11.part
12.part
13.part
14.part
15.part
16.part
17.part
18.part
19.part
20.part
21.part
22.part
23.part
24.part
25.part
26.part
27.part
28.part
29.part
30.part
31.part
32.part
33.part
34.part
35.part
36.part
37.part
38.part
39.part
40.part
41.part
42.part
43.part


In [32]:
os.path.getsize('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week 6/Books_rating.csv.gz')    

8192

In [35]:
print('Number of Columns:')
print(df.shape[1])

Number of Columns:
10


In [37]:
print('Number of rows:')
print(len(df.index))

Number of rows:
3000000
